In [1]:
# ------------------------------
#  Imports et setup
# ------------------------------
import re
import nltk
import pandas as pd
from datasets import Dataset, concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import LoraConfig, get_peft_model

nltk.download('punkt')

# ------------------------------
#  Paramètres
# ------------------------------
model_name = "t5-small"      # plus léger pour Kaggle
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

chunksize = 10000            # nombre de lignes par chunk pour charger
file_path = '/kaggle/input/c4200m/C4_200M.tsv-00000-of-00010'  # chemin vers ton fichier

# ------------------------------
#  Fonction de nettoyage
# ------------------------------
def clean_text(text):
    if text is None:
        return ""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# ------------------------------
# Préprocessing pour tokenisation
# ------------------------------
def preprocess(examples):
    inputs = examples.get("text", [])
    cleans = examples.get("clean", [])
    
    # Nettoyage
    cleaned = [clean_text(c) for c in cleans]
    
    # Tokenisation
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(cleaned, max_length=128, truncation=True, padding="max_length")
    
    # Masque pour la perte
    labels_ids = [[l if l != tokenizer.pad_token_id else -100 for l in label] for label in labels["input_ids"]]
    model_inputs["labels"] = labels_ids
    return model_inputs

# ------------------------------
#  Chargement dataset par chunks
# ------------------------------
dataset_chunks = []

for chunk in pd.read_csv(file_path, sep='\t', header=None, names=["text", "clean"], chunksize=chunksize):
    chunk = chunk.fillna({"text": "", "clean": ""})
    
    # 🔹 Filtrage sur ratio de longueur
    def length_filter(row, max_ratio=2.0):
        text_len = len(row["text"].split())
        clean_len = len(row["clean"].split())
        if text_len == 0 or clean_len == 0:
            return False
        ratio = max(text_len, clean_len) / min(text_len, clean_len)
        return ratio <= max_ratio
    
    chunk = chunk[chunk.apply(length_filter, axis=1)]
    
    ds_chunk = Dataset.from_pandas(chunk)
    dataset_chunks.append(ds_chunk)

# Concaténer tous les chunks
dataset = concatenate_datasets(dataset_chunks)
dataset=dataset.select(range(100000))
print("Dataset loaded:", len(dataset))

# ------------------------------
#  Séparer train/test
# ------------------------------
train_limit = int(len(dataset) * 0.8)
train_dataset = dataset.select(range(train_limit))
test_dataset = dataset.select(range(train_limit, len(dataset)))

# ------------------------------
#  Tokenisation par chunks
# ------------------------------
def tokenize_dataset(dataset, batch_size=500):
    tokenized_chunks = []
    for i in range(0, len(dataset), batch_size):
        chunk = dataset.select(range(i, min(i+batch_size, len(dataset))))
        tokenized_chunk = chunk.map(preprocess, batched=True)
        tokenized_chunks.append(tokenized_chunk)
    return concatenate_datasets(tokenized_chunks)

print("Tokenizing train dataset...")
tokenized_train = tokenize_dataset(train_dataset, batch_size=500)

print("Tokenizing test dataset...")
tokenized_test = tokenize_dataset(test_dataset, batch_size=500)

# ------------------------------
#  LoRA config (optionnel)
# ------------------------------
config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

# Appliquer LoRA après test
model = get_peft_model(model, config)

print("Pipeline ready. Train and test datasets tokenized.")


2025-10-18 14:01:34.429122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760796094.619497      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760796094.687209      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Pipeline ready. Train and test datasets tokenized.


In [2]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    output_dir="./out",
    num_train_epochs=19,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-3,                   
    lr_scheduler_type="linear",           
    warmup_steps=200,                     
    weight_decay=0.01,
    predict_with_generate=True,
    #evaluation_strategy="epoch",          
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

trainer.train()


No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,1.235300
100,1.068300
150,0.936300
200,0.947200
250,0.923700
300,0.836700
350,0.875600
400,0.878600
450,0.890800
500,0.883200


TrainOutput(global_step=190000, training_loss=0.773894047415884, metrics={'train_runtime': 15940.4771, 'train_samples_per_second': 95.355, 'train_steps_per_second': 11.919, 'total_flos': 5.2118421504e+16, 'train_loss': 0.773894047415884, 'epoch': 19.0})

In [3]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [4]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.6 MB/s eta 0:00:00


In [5]:
import torch


import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.translate.bleu_score import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def correct_sentences(sentences, batch_size=8):
    all_preds = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        inputs = tokenizer(list(batch), return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=64)  # réduit max_length
        preds = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]
        all_preds.extend(preds)
        torch.cuda.empty_cache()
    return all_preds


#  Fonction d’évaluation

def evaluate(df, dataset_name="dataset"):
    # 🔹 Décoder les labels
    decoded_refs = []
    for label_ids in df["labels"]:
        # enlever les -100 (padding labels)
        label_ids_clean = [id for id in label_ids if id != -100]
        text = tokenizer.decode(label_ids_clean, skip_special_tokens=True)
        decoded_refs.append(text)

    #  Prédictions
    decoded_preds = correct_sentences(df["text"])  # ⚠️ adapter à ta colonne input texte

    #  Préparer BLEU
    refs_bleu = [[ref.split()] for ref in decoded_refs]
    preds_bleu = [pred.split() for pred in decoded_preds]
    bleu_score = corpus_bleu(refs_bleu, preds_bleu)
    
    #  Affichage
    print(f"\n Résultats sur {dataset_name}:")
    print(f"BLEU       : {bleu_score:.4f}")
   

    return decoded_preds, decoded_refs



preds, refs = evaluate(tokenized_test, "Test")

# Vérification des  quelques exemples
for p, r in zip(preds[:5], refs[:5]):
    print("\nPrédiction :", p)
    print("Référence  :", r)


 Résultats sur Test:
BLEU       : 0.6003

Prédiction : The new symposium came after a negotiating party met several marathon bargaining sessions this week, with the closing of doors in Newark.
Référence  : The new agreement came after negotiators met for several marathon bargaining sessions this week behind closed doors in Newark.

Prédiction : The wrote appeal's target at length is the accusation of terrorism and spying against Brunson, which based on the testimony of a key “secret” witness code named Dua.
Référence  : The written appeal targets at length the accusations of terrorism and espionage against Brunson which are based on the testimony of a key “secret” witness code-named “Dua”.

Prédiction : Data protection impact assessment – this will be carried out when using new technologies, and the processing is likely to result in a high risk for the rights and freedom of individuals of their language.
Référence  : Data protection impact assessment – this will be carried out when us

In [6]:
import torch
import numpy as np
from sacrebleu.metrics import CHRF

# 🔹 Préparation metric
chrf_metric = CHRF()

# 🔹 Listes pour stocker
predictions = []
references = []

# 🔹 Boucle par batch
batch_size = 8
for i in range(0, len(tokenized_test), batch_size):
    batch = tokenized_test.select(range(i, min(i+batch_size, len(tokenized_test))))
    
    input_ids = torch.tensor(np.array(batch["input_ids"])).to(model.device)
    attention_mask = torch.tensor(np.array(batch["attention_mask"])).to(model.device)
    
    # Labels (pour récupérer références)
    labels = np.array(batch["labels"])
    
    # 🔹 Génération des prédictions
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids,
                                 attention_mask=attention_mask,
                                 max_length=128)
    
    # 🔹 Décodage des prédictions
    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # 🔹 Décodage des labels (références)
    # Remplacer -100 (masque) par pad_token_id pour décoder proprement
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    predictions.extend(decoded_preds)
    references.extend(decoded_refs)

# 🔹 Calcul CHRF
chrf_score = chrf_metric.corpus_score(predictions, [references]).score
print(f"CHRF++ sur tokenized_test : {chrf_score:.2f}")


CHRF++ sur tokenized_test : 82.99
